In [ ]:
from model import atp_graph, losses
from Tutorials.helper import batcher
from data import synthetic_data_gen, feature_extractor
import keras
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import tensorflow as tf
import IPython
import time
import os

from model import atp_pipeline

In [ ]:
from data import dataset_preparer

#dataset - [weather,traffic,...]

#hyperparams - num_heads,...

In [ ]:
weather_processor = dataset_preparer.weather(path_to_weather_data="datasets/weather.csv")

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = weather_processor()

In [ ]:
save_dir = "weights/forecasting/weather"

In [ ]:
num_heads = 4

In [ ]:
tf.random.set_seed(442)
opt = tf.keras.optimizers.Adam(3e-4)
run=3; heads=2
atp_model = atp_pipeline.atp_pipeline(num_heads=num_heads)
EPOCHS = 2

In [ ]:
seq_l = 96+192;
num_batches = 500
batch_s=20
mini = 50000
n_C_te =5

In [ ]:
step = 1
tr_step = atp_graph.build_graph()

name_comp = 'run_' + str(run)
logdir = save_dir + '/logs/' + name_comp
folder = save_dir + '/ckpt/check_' + name_comp

if not os.path.exists(folder): os.mkdir(folder)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)

In [ ]:
for epoch in range(2):
    print("epoch: ",+epoch)
    start = time.time()
    for batch_n in range(10):
         
        x,y = batcher(training_data_scaled[:,:1],training_data_scaled[:,-2:])
        n_C = int(np.random.choice(np.arange(2, 20), 1))
        n_T = 288 - n_C
        y_tr = y
        t_tr = x
        _,_, nll_pp, msex = tr_step(atp_model, opt, t_tr,y_tr,n_C,n_T, training=True)

        if ((batch_n % 2 == 0)):
            
            n_C = 10
            n_T = 200 - n_C
            t_te,y_te = batcher(val_data_scaled[:,:1], val_data_scaled[:,-2:],batch_s = 100)
            μ, log_σ = atp_model([t_te, y_te, 10, 200, False])
            _,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ[:, n_C:], log_σ[:, n_C:])
            
        if nll_pp_te < mini:
            mini = nll_pp_te
            IPython.display.clear_output(wait=True)
            print("epoch {} batch {} validation lik pp: {}".format(epoch, batch_n, nll_pp)) 
            manager.save()
            step += 1
            ckpt.step.assign_add(1)
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

In [ ]:
from model import testing

In [ ]:
testing.testing_multiple_times([training_data_scaled, test_data_scaled], save_dir)

In [ ]:
        
        training_data_scaled, test_data_scaled = inputs
        #instantiation
        
        step = 1
        run=run_num; 
        tf.random.set_seed(run)
        opt = tf.keras.optimizers.Adam(3e-4)
        atp_model = atp_pipeline.atp_pipeline( num_heads=num_heads, projection_shape_for_head=projection_shape_for_head, output_shape=output_shape, rate=rate, permutation_repeats=permutation_repeats,
        bound_std=bound_std, num_layers=num_layers, enc_dim=enc_dim,  xmin=xmin, xmax=xmax)
        tr_step = atp_graph.build_graph()
        name_comp = 'run_' + str(run)
        folder = save_dir + '/ckpt/check_' + name_comp
        if not os.path.exists(folder): os.mkdir(folder)
        ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model)
        manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
        ckpt.restore(manager.latest_checkpoint) 
        sum_mse_tot = 0; sum_nll_tot = 0
        mini = =50000
        
        #training
        for epoch in range(num_epochs):
            print("epoch: ",+epoch)
            for _ in range(num_batches):
                x,y = batcher(training_data_scaled[:,:1],training_data_scaled[:,-2:])
                n_C = int(np.random.choice(np.arange(2, 20), 1))
                n_T = 288 - n_C
                _,_, _, _ = tr_step(atp_model, opt, x,y,n_C,n_T, training=True)
                
                #saving 
                
                if nll_pp_te < mini:
                    mini = nll_pp_te
                    manager.save()
                    step += 1
                    ckpt.step.assign_add(1)
        
        ckpt.restore(manager.latest_checkpoint) 


        for _ in range(test_data_scaled.shape[0] // test_batch_s):
            n_C = 10
            n_T = 200 - n_C
            t_te,y_te = batcher(test_data_scaled[:,:1], test_data_scaled[:,-2:], batch_s = test_batch_s)
            μ, log_σ = atp_model([t_te,y_te,10,200, False])
            _, sum_mse, sum_nll, _, _ = losses.nll(y_te[:, n_C:n_C+n_T], μ[:, n_C:], log_σ[:, n_C:])
            sum_nll_tot += sum_nll # each sequence is length 190 times 100 seq. per batch 
            sum_mse_tot += sum_mse
            
        nllx =  sum_nll_tot / (n_T * test_batch_s * (test_data_scaled.shape[0] // test_batch_s))
        msex =  sum_mse_tot / (n_T * test_batch_s * (test_data_scaled.shape[0] // test_batch_s))
        
            
        nll_list.append(nllx.numpy())
        mse_list.append(msex.numpy())
    np.save(save_dir + '/nll_list.npy', nll_list)    
    np.save(save_dir + '/mse_list.npy', mse_list)   

In [ ]:
####

#electricity

# make batcher as a function of n_C and n_T - window

## other NP experiments

# test on n_C and n_T which are randomly selected at each point - outside batcher 

# don't repeat testing points  

# for NP experiments, we need to batch differently. for forecasting 100000 x1 ---> 32 x 288 x 1.
# for np, we will have 10000 x 288 x 1 -> 32 x 288 x 1

#also divide mse by n_T each batch for testing

In [ ]:
### set this up for training and saving best models adn then loading it and calculating loglik.

In [ ]:
def batcher(t, y, idx_list,batch_s = 32, window = 288,cut_sequence = True):
    '''
    cutting one long array to sequences of length 'window'.
    'batch_s' must be ≤ full array - window length
    '''
    
    
    if len(idx_list) < 1:
        print("warning- you didn't loop over the correct range")
        
    
    batch_s = min(batch_s, y.shape[0]-window)    
    idx = np.random.choice(len(idx_list), batch_s, replace = False)

    y = np.array([np.array(y)[idx_list[i]:idx_list[i]+window] for i in idx])
    t = np.array([np.array(t)[idx_list[i]:idx_list[i]+window] for i in idx])
    for i in sorted(idx, reverse=True): del idx_list[i]
    return t, y,idx_list

In [ ]:
y_tr = Y[tr_ix, :, :]
t_tr = t[tr_ix, :]

In [ ]:
def batcher(t, y, idx_list, batch_s = 32, window = 288):
    '''
    cutting one long array to sequences of length 'window'.
    'batch_s' must be ≤ full array - window length

    input to forecast: (None, 1, 1) for t,y.
    input to NP tasks: (None, seq_len, 1) for t,y.
    '''
    
    
    if len(idx_list) < 1:
        print("warning- you didn't loop over the correct range")
        
    
    batch_s = min(batch_s, y.shape[0]-window)    
    idx = np.random.choice(len(idx_list), batch_s, replace = False)

    y = np.array([np.array(y)[idx_list[i]:idx_list[i]+window, :, :] for i in idx])
    t = np.array([np.array(t)[idx_list[i]:idx_list[i]+window, :, :] for i in idx])
    for i in sorted(idx, reverse=True): del idx_list[i]
        
    t = t.squeeze()
    y = y.squeeze()
    
    if len(t.shape) == 2:
        t = t[:,:,np.newaxis]
        y = y[:,:,np.newaxis]
        
    return t,y, idx_list

In [ ]:
idx_list=list(range(y_test.shape[0]-288))

In [ ]:
a,b,c = batcher(x_test[:,:,np.newaxis],y_test[:,:,np.newaxis],idx_list=idx_list)

In [ ]:
x_dummy = np.random.normal(size=(100,300,1))

In [ ]:
y_dummy = np.random.normal(size=(100,300,1))

In [ ]:
idx_list=list(range(y_dummy.shape[0]))

In [ ]:
y_dummy.shape

In [ ]:
a,b,c = batcher(x_dummy,y_dummy,idx_list=idx_list,window=1)

## start here

In [ ]:
from model import atp_graph, losses
from data import synthetic_data_gen, feature_extractor
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from model import atp_pipeline
from data import dataset_preparer
import argparse
from Tutorials.helper import batcher
import os

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset_preparer.weather_processor(path_to_weather_data="datasets/weather.csv") 
save_dir = "weights/forecasting/weather"

In [ ]:
n_C = 100
n_T = 100

opt = tf.keras.optimizers.Adam(3e-4)
batch_size = 32

In [ ]:
idx_list = list(range(x_train.shape[0] - (n_C+n_T)))
x, y, _ = batcher(x_train, y_train, idx_list, window=n_C+n_T)

In [ ]:
def batcher_x(t, y, idx_list, batch_s = 32, window = 288):
    '''
    cutting one long array to sequences of length 'window'.
    'batch_s' must be ≤ full array - window length

    input to forecast: (None, 1, 1) for t,y.
    input to NP tasks: (None, seq_len, 1) for t,y. window = 1.
    '''
    
    
    if len(idx_list) < 1:
        print("warning- you didn't loop over the correct range")
        
    
    batch_s = min(batch_s, y.shape[0]-window)    
    idx = np.random.choice(len(idx_list), batch_s, replace = False)

    print(idx)
    y = np.array([np.array(y)[idx_list[i]:idx_list[i]+window, :, :] for i in idx])
    t = np.array([np.array(t)[idx_list[i]:idx_list[i]+window, :, :] for i in idx])
    print(t.shape)
    for i in sorted(idx, reverse=True): del idx_list[i]
        
    t = t.squeeze()
    y = y.squeeze()
    
    if len(t.shape) == 2:
        t = t[:,:,np.newaxis]
        y = y[:,:,np.newaxis]
        
    return t,y, idx_list

In [ ]:
idx_list_v = list(range(x_val.shape[0] - (n_C+n_T)))
t_te,y_te,_ = batcher_x(x_val, y_val, idx_list_v, batch_s = 100, window=(n_C+n_T))

In [ ]:
step = 1
run= 50 + 0
tf.random.set_seed(run)
atp_model = atp_pipeline.instantiate_atp("weather")
tr_step = atp_graph.build_graph()
name_comp = 'run_' + str(run)
folder = save_dir + '/ckpt/check_' + name_comp
if not os.path.exists(folder): os.mkdir(folder)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint) 
sum_mse_tot = 0; sum_nll_tot = 0
mini = 50000

for i in range(2):
    idx_list = list(range(x_train.shape[0]))
    x,y,_ = batcher(x_train,y_train,idx_list,window=n_C+n_T) ####### generalise for not weather
    _,_, _, _ = tr_step(atp_model, opt, x,y,n_C,n_T, training=True)

    if i % 1 == 0:
        idx_list = list(range(x_val.shape[0]))
        t_te,y_te,_ = batcher(x_val,y_val,idx_list,batch_s = 100,window=n_C+n_T)
        μ, log_σ = atp_model([t_te, y_te, n_C, n_T, False])
        _,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)

        if nll_pp_te < mini:
            mini = nll_pp_te
            manager.save()
            step += 1
            ckpt.step.assign_add(1)

In [ ]:
x_train.shape

In [ ]:
x_val.shape

In [ ]:
t_te,y_te,_ = batcher(x_train,y_train,idx_list,batch_s = 32,window=n_C+n_T)


In [ ]:
t_te.shape

In [ ]:
t_te.shape

In [ ]:
x_train.shape